In [45]:
import numpy as np
import torch

In [46]:
def myConv2d(
  matrix,
  in_channels,
  out_channels,
  kernel_size,
  stride=1,
  padding=0,
  dilation=1,
  groups=1,
  bias=True,
  padding_mode='zeros'
  ):
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(2)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    if type(kernel_size) == tuple:
      weight = torch.rand(
        out_channels,
        in_channels//groups,
        kernel_size[0],
        kernel_size[1],
        )
    if type(kernel_size) == int:
      weight = torch.rand(
        out_channels,
        in_channels//groups,
        kernel_size,
        kernel_size,
        )
    result = []
    for l in range(out_channels):
      feature_map = np.array([])
      for i in range (0, matrix.shape[1]-((weight.shape[2]-1)*dilation+1)+1, stride):
        for j in range (0, matrix.shape[2]-((weight.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels//groups):
            if groups>1:
              val = matrix[l*(in_channels//groups)+c][
                i:i+(weight.shape[2]-1)*dilation+1:dilation,
                j:j+(weight.shape[3]-1)*dilation+1:dilation,
                ]
            else:
              val = matrix[c][
                i:i+(weight.shape[2]-1)*dilation+1:dilation,
                j:j+(weight.shape[3]-1)*dilation+1:dilation,
                ]
            mini_sum = (val*weight[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l]))

      result.append(
        feature_map.reshape(
          (matrix.shape[1]-((weight.shape[2]-1)*dilation+1))//stride+1,
          (matrix.shape[2]-((weight.shape[3]-1)*dilation+1))//stride+1,
          ))

    return np.array(result), np.array(weight), np.array(bias_val),


In [47]:
tensor1 = torch.rand(4, 5, 6)
tensor2 = torch.rand(3, 28, 28)
tensor3 = torch.rand(1, 1, 1)

In [48]:
result, weight, bias_val = myConv2d(
    tensor1,
    in_channels=4,
    out_channels=2,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    groups=2,
    bias=True,
    padding_mode='zeros',
    )
torchFunction = torch.nn.Conv2d(
    in_channels=4,
    out_channels=2,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    groups=2,
    bias=True,
    padding_mode='zeros',
    )
torchFunction.weight.data = torch.tensor(weight)
torchFunction.bias.data = torch.tensor(bias_val)
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
torchResult == myResult


True

In [49]:
result2, weight2, bias_val2 = myConv2d(
    tensor2, in_channels=3, 
    out_channels=1, 
    kernel_size=3, 
    stride=1, 
    padding=0, 
    dilation=2, 
    groups=1, 
    bias=True, 
    padding_mode='zeros',
    )
torchFunction2 = torch.nn.Conv2d(
    in_channels=3, 
    out_channels=1, 
    kernel_size=3, 
    stride=1, 
    padding=0, 
    dilation=2, 
    groups=1, 
    bias=True, 
    padding_mode='zeros',
    )
torchFunction2.weight.data = torch.tensor(weight2)
torchFunction2.bias.data = torch.tensor(bias_val2)
myResult = str(np.round(result2, 2))
torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
torchResult == myResult

True

In [50]:
result3, weight3, bias_val3 = myConv2d(
    tensor3, 
    in_channels=1, 
    out_channels=1, 
    kernel_size=1, 
    stride=1, 
    padding=0, 
    dilation=1, 
    groups=1, 
    bias=True, 
    padding_mode='zeros',
    )
torchFunction3 = torch.nn.Conv2d(
    in_channels=1, 
    out_channels=1, 
    kernel_size=1, 
    stride=1, 
    padding=0, 
    dilation=1, 
    groups=1, 
    bias=True, 
    padding_mode='zeros',
    )
torchFunction3.weight.data = torch.tensor(weight3)
torchFunction3.bias.data = torch.tensor(bias_val3)
myResult = str(np.round(result3, 2))
torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
torchResult == myResult

True